In [1]:
import os
os.getcwd()

'C:\\Users\\Carl\\Housing'

In [2]:
os.chdir("C:\\Users\\Carl\\Housing")

## Read Postcodewithtimedf to get postcode per LA

In [3]:
import pandas as pd
import statistics
import os
from operator import *
pd.set_option('display.max_rows', None)
os.getcwd()

'C:\\Users\\Carl\\Housing'

In [4]:
columns=["Transaction unique identifier","Price","Date of Transfer","Postcode","Property Type","Old/New","Duration","PAON","SAON","Street","Locality","Town/City","District","County","PPD Category Type","TBD"]

print("postcode with train times starting...")
postcodes_with_train_times_df=pd.read_excel("C:\\Users\\Carl\\Housing\\postcodedfwithtimedf.xlsx")
postcodes_with_train_times_df=postcodes_with_train_times_df[["station_name","postcode","latitude","longitude","train time to work","First Postcode"]]
print("postcode with train times done")


print("price paid data starting...")
pp2022=pd.read_csv("C:\\Users\\Carl\\Housing\\Data\\WIMD Data\\pp-2022.csv")
pp2022.columns=columns
print("price paid data done")



print("postcode to LA data starting...")
postcodemappings=pd.read_csv(open("C:\\Users\\Carl\\Housing\\Data\\Postcode to LA.csv"),sep=',',error_bad_lines=False)
postcodemappings["Local Authority Name"]=postcodemappings["Local Authority Name"].str.rstrip()
postcodemappings["Local Authority Name"]=postcodemappings["Local Authority Name"].str.lstrip()
postcodemappings["Local Authority Name"]=postcodemappings["Local Authority Name"].str.upper()
print("postcode to LA done")

print("wimd starting...")
wimd=pd.read_csv(open(str(os.getcwd())+"\\Data\\WIMD Data\\WIMD 2019 by Rank (LSOA).csv"),skiprows=range(0,5))
wimd["Unnamed: 0"]=wimd["Unnamed: 0"].fillna(method="ffill")
wimd=wimd.rename(columns={"Unnamed: 0":"Local Authority Name","Unnamed: 1":"LSOA Name"})
wimd["Local Authority Name"]=wimd["Local Authority Name"].str.rstrip()
wimd["Local Authority Name"]=wimd["Local Authority Name"].str.lstrip()
wimd["Local Authority Name"]=wimd["Local Authority Name"].str.upper()
wimd["LSOA Name"]=wimd["LSOA Name"].str.rstrip()
wimd["LSOA Name"]=wimd["LSOA Name"].str.lstrip()
wimd["LSOA Name"]=wimd["LSOA Name"].str.upper()
wimd=wimd.drop(columns=["Unnamed: 2"])
wimd=wimd[1:1931]
wimd=wimd[wimd['WIMD ']!="."]
wimd["WIMD "]=wimd["WIMD "].astype(int)
wimd["Income "]=wimd["Income "].astype(int)
wimd["WIMD "]=wimd["WIMD "].astype(int)
wimd["Employment "]=wimd["Employment "].astype(int)
wimd["Health "]=wimd["Health "].astype(int)
wimd["Education "]=wimd["Education "].astype(int)
wimd["Access to Services "]=wimd["Access to Services "].astype(int)
wimd["Community Safety "]=wimd["Community Safety "].astype(int)
wimd["Community Safety "]=wimd["Community Safety "].astype(int)
wimd["Physical Environment "]=wimd["Physical Environment "].astype(int)
wimd["Housing "]=wimd["Housing "].astype(int)
deprivation_scores=wimd
print("wimd done")


print("retail info starting...")
retail_info=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\RetailInfo.xlsx','rb'),"Table 1",usecols=[1,7],skiprows=range(5))
retail_info=retail_info.dropna()
retail_info=retail_info.rename(columns={"Unnamed: 1":"Local Authority","47 : Retail":"Count"})
retail_info["Local Authority"]=retail_info["Local Authority"].str.rstrip()
retail_info["Local Authority"]=retail_info["Local Authority"].str.lstrip()
retail_info["Local Authority"]=retail_info["Local Authority"].str.upper()
retail_info.set_index("Local Authority",inplace=True)
retail_infodict=retail_info.to_dict(orient="index")
print("retail info done")


print("crime info starting...")
crimedf=pd.read_excel(open("C:\\Users\\Carl\\Housing\\Data\\Crime Data.xlsx","rb"),"Table C3",skiprows=range(6))
crimedf.columns=[string.replace("\n","") for string in list(crimedf.columns)]
crimedf=crimedf[["Local Authority name","Total recorded crime (excluding fraud)"]]
crimedf["Local Authority name"]=crimedf["Local Authority name"].str.rstrip()
crimedf["Local Authority name"]=crimedf["Local Authority name"].str.lstrip()
crimedf["Local Authority name"]=crimedf["Local Authority name"].str.upper()
crimedf.dropna(subset=["Local Authority name"],inplace=True)
crimedf=crimedf.set_index("Local Authority name")
crimedfdict=crimedf.to_dict(orient="index")
print("crime info done")

print("population info starting...")
population_info=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\Population Figures (england and wales).xlsx','rb'),"P01",usecols=[1,2],skiprows=range(6))
population_info=population_info.dropna()
population_info=population_info.rename(columns={"Area name":"Local Authority","All persons":"Population"})
population_info["Local Authority"]=population_info["Local Authority"].str.rstrip()
population_info["Local Authority"]=population_info["Local Authority"].str.lstrip()
population_info["Local Authority"]=population_info["Local Authority"].str.upper()
population_info.set_index("Local Authority",inplace=True)
population_infodict=population_info.to_dict(orient="index")
print("population info done")

print("area size info starting...")
areasizeinfo=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\SAM_LAD_DEC_2020_UK.xlsx','rb'),"SAM_LAD_2020",usecols=[1,2])
areasizeinfo=areasizeinfo.dropna()
areasizeinfo.rename(columns={"LAD20NM":"Local Authority","AREAEHECT":"Area (Hectares)"},inplace=True)
areasizeinfo["Local Authority"]=areasizeinfo["Local Authority"].str.rstrip()
areasizeinfo["Local Authority"]=areasizeinfo["Local Authority"].str.lstrip()
areasizeinfo["Local Authority"]=areasizeinfo["Local Authority"].str.upper()
areasizeinfo.set_index("Local Authority",inplace=True)
areasizeinfodict=areasizeinfo.to_dict(orient="index")
print("area size info done")


print("getting the count of beaches per first postcode...")

beachdf=pd.read_excel(open("C:\\Users\\Carl\\Housing\\Data\\Beaches and Bathing Sites in Wales\\Welsh Beaches with Long and Lats.xlsx","rb"))
beachdf=beachdf.rename(columns={"Unnamed: 0":"Beach Name"})
beachcountdf=beachdf["First Postcode"].value_counts().to_frame()
beachcountdict=beachcountdf.to_dict()
beachcountdict=list(beachcountdict.values())[0]
beachcountdict

print("beach dict done")


print("Green Data starting...")
welshgreendf=pd.read_excel("C:\\Users\\Carl\\Housing\\Data\\WIMD Data\\Green Data for Welsh LA.xlsx")
welshgreendf=welshgreendf.rename(columns={"Unnamed: 0":"Local Authority"})
welshgreendf=welshgreendf.set_index("Local Authority")
print("Green Data finished...")

print("all done")

### THEN PUT IN A DICT, AND FILTER ALL POSTCODES IN WALES TO SEE IF THEY MATCH
### IF NOT THEN INDICATE AS SUCH ON THE FINAL DF AND ALTER SCORE

postcode with train times starting...
postcode with train times done
price paid data starting...
price paid data done
postcode to LA data starting...
postcode to LA done
wimd starting...
wimd done
retail info starting...
retail info done
crime info starting...
crime info done
population info starting...
population info done
area size info starting...
area size info done
getting the count of beaches per first postcode...
beach dict done
Green Data starting...
Green Data finished...
all done


In [5]:
waleslalist=wimd["Local Authority Name"].value_counts().keys().tolist()

In [6]:
import re
index=1
trimdict={}
for postcode,count in beachcountdict.items():
    postcodestem=re.sub(r'\d+', '', postcode)
    #print(postcode,postcodestem,count)
    trimdict[index]=[postcodestem,count]
    index+=1
beachcountbypostcodestem=pd.DataFrame.from_dict(trimdict,orient="index").groupby([0]).sum()
beachcountbypostcodestem=list(beachcountbypostcodestem.to_dict().values())[0]
beachcountbypostcodestem

{'CCV': 1, 'CF': 8, 'CQ': 1, 'LL': 40, 'SA': 49, 'SY': 5}

In [7]:
postcodes_with_train_times_df.head()

,station_name,postcode,latitude,longitude,train time to work,First Postcode
0,Alexandra Palace Rail Station,N22 7SS,51.597925,-0.120210,44.966667,N22
1,Achanalt Rail Station,IV23 2QD,57.609576,-4.913846,957.300000,IV23
2,Aberdare Rail Station,CF44 7BP,51.715060,-3.443095,436.133333,CF44
3,Altnabreac Rail Station,KW12 6UR,58.388133,-3.706287,1055.300000,KW12
4,Aberdeen Rail Station,AB11 6LX,57.143687,-2.098693,698.300000,AB11


In [8]:
pp2022.head()

,Transaction unique identifier,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,Street,Locality,Town/City,District,County,PPD Category Type,TBD
0,{DBA933FA-5666-669D-E053-6B04A8C0AD56},367250,2022-02-25 00:00,HU17 9RT,D,N,F,"ASHCROFT, 216",NaN,HULL BRIDGE ROAD,NaN,BEVERLEY,EAST RIDING OF YORKSHIRE,EAST RIDING OF YORKSHIRE,A,A
1,{DBA933FA-5668-669D-E053-6B04A8C0AD56},425000,2022-02-18 00:00,HU14 3EZ,D,N,F,4,NaN,PARKLANDS CRESCENT,NaN,NORTH FERRIBY,EAST RIDING OF YORKSHIRE,EAST RIDING OF YORKSHIRE,A,A
2,{DBA933FA-566B-669D-E053-6B04A8C0AD56},217000,2022-02-18 00:00,DN14 7ZL,S,N,F,59,NaN,LANGHORN DRIVE,HOWDEN,GOOLE,EAST RIDING OF YORKSHIRE,EAST RIDING OF YORKSHIRE,A,A
3,{DBA933FA-566E-669D-E053-6B04A8C0AD56},189950,2022-01-31 00:00,YO16 6AE,D,N,F,6,NaN,ALDERSHOT DRIVE,NaN,BRIDLINGTON,EAST RIDING OF YORKSHIRE,EAST RIDING OF YORKSHIRE,A,A
4,{DBA933FA-56D4-669D-E053-6B04A8C0AD56},540000,2022-03-04 00:00,YO43 4DU,D,N,F,LILAC COTTAGE,NaN,SANDHOLES,HOLME ON SPALDING MOOR,YORK,EAST RIDING OF YORKSHIRE,EAST RIDING OF YORKSHIRE,A,A


In [9]:
postcodemappings.head()

,ï»¿Postcode 1,Postcode 2,Postcode 3,County Code,County Name,Local Authority Code,Local Authority Name
0,OX277JB,OX27 7JB,OX27 7JB,E10000025,Oxfordshire,E07000177,CHERWELL
1,NR205NQ,NR20 5NQ,NR20 5NQ,E10000020,Norfolk,E07000143,BRECKLAND
2,GU214YG,GU21 4YG,GU21 4YG,E10000030,Surrey,E07000217,WOKING
3,GU220AJ,GU22 0AJ,GU22 0AJ,E10000030,Surrey,E07000217,WOKING
4,DA8 1DN,DA8 1DN,DA8 1DN,E13000002,Outer London,E09000004,BEXLEY


In [10]:
deprivation_scores.head()

,Local Authority Name,LSOA Name,WIMD,Income,Employment,Health,Education,Access to Services,Community Safety,Physical Environment,Housing
1,ISLE OF ANGLESEY,AMLWCH PORT,550,414,420,676,615,767,667,1776,766
2,ISLE OF ANGLESEY,AMLWCH RURAL,1100,1122,1004,1311,787,265,1284,1827,1229
3,ISLE OF ANGLESEY,LLANBADRIG,1145,759,815,1159,972,1345,1337,1829,1081
4,ISLE OF ANGLESEY,LLANNERCH-Y-MEDD,727,858,940,1090,865,90,1593,1318,237
5,ISLE OF ANGLESEY,MECHELL,864,878,842,1322,1194,131,1683,1700,305


In [11]:
retail_info.head(20)

,Count
Local Authority,
UNITED KINGDOM,220685.0
GREAT BRITAIN,214865.0
ENGLAND AND WALES,201270.0
ENGLAND,192870.0
NORTH EAST,6090.0
COUNTY DURHAM,1220.0
DARLINGTON,300.0
HARTLEPOOL,190.0
MIDDLESBROUGH,280.0


In [12]:
population_info.head(20)

,Population
Local Authority,
ENGLAND AND WALES,59597300
ENGLAND,56489800
NORTH EAST,2647100
COUNTY DURHAM,522100
DARLINGTON,107800
HARTLEPOOL,92300
MIDDLESBROUGH,143900
NORTHUMBERLAND,320600
REDCAR AND CLEVELAND,136500


In [13]:
areasizeinfo.head(20)

,Area (Hectares)
Local Authority,
HARTLEPOOL,9834.66
MIDDLESBROUGH,5455.37
REDCAR AND CLEVELAND,25378.57
STOCKTON-ON-TEES,20973.09
DARLINGTON,19747.58
HALTON,9032.15
WARRINGTON,18238.34
BLACKBURN WITH DARWEN,13702.21
BLACKPOOL,4315.08


In [15]:
beachcountdict

{'SA62': 11,
 'SA3': 7,
 'SA70': 6,
 'LL53': 6,
 'SA71': 5,
 'SY23': 4,
 'SA43': 4,
 'LL18': 4,
 'CF36': 3,
 'LL65': 3,
 'SA45': 3,
 'CF62': 3,
 'SA69': 3,
 'SA44': 3,
 'LL46': 2,
 'LL30': 2,
 'LL35': 2,
 'LL42': 2,
 'CF32': 1,
 'SA73': 1,
 'LL34': 1,
 'LL70': 1,
 'SA16': 1,
 'SA1': 1,
 'SA42': 1,
 '9CCV': 1,
 'LL19': 1,
 'SA67': 1,
 'LL58': 1,
 'LL49': 1,
 'LL64': 1,
 'LL74': 1,
 'LL61': 1,
 'LL38': 1,
 'LL75': 1,
 'SA12': 1,
 'CF61': 1,
 'LL36': 1,
 'SY24': 1,
 'LL52': 1,
 'LL55': 1,
 'LL67': 1,
 'LL63': 1,
 'LL22': 1,
 '7C2Q': 1,
 'LL28': 1,
 'SA33': 1,
 'LL33': 1,
 'LL54': 1}

In [14]:
round(statistics.median(list(beachcountdict.values())))

1

# Flat calculations

In [16]:
pd.set_option('display.max_rows', None)
#pp2020=pd.read_csv("C:\\Users\\Carl\\Housing\\Data\\pp-2020.csv")
#pp2020.columns=columns
#pp2021=pd.read_csv("C:\\Users\\Carl\\Housing\\Data\\pp-2021.csv")
#pp2021.columns=columns

print("shape of input df is",pp2022.shape)

pp2022['Property Type']=pp2022["Property Type"].str.replace("D","Detached")
pp2022['Property Type']=pp2022["Property Type"].str.replace("S","Semi-Detached")
pp2022['Property Type']=pp2022["Property Type"].str.replace("T","Terraced")
pp2022['Property Type']=pp2022["Property Type"].str.replace("F","Flat")
pp2022["Property Type"]=pp2022["Property Type"].str.replace("O","Other")

print("\nthis is the breakdown by house type across all UK\n",pp2022["Property Type"].value_counts())

hpp2022=pp2022[pp2022["Property Type"].isin(["Flat"])]
print("\n houses sold since 2022\n",hpp2022.shape)

hpp2022.reset_index(inplace=True,drop=True)

hpp2022.dropna(subset=["Postcode"],inplace=True)
hpp2022.reset_index(inplace=True,drop=True)

hpp2022dict=hpp2022.to_dict(orient="index")
print("hpp2022dict[2]=",hpp2022dict[2])

for i in range(0,len(hpp2022)):
    try:
        postcode=list(hpp2022dict.get(i).items())[3][1]
        first_postcode=postcode.split(" ")[0]
        hpp2022dict[i]["First Postcode"]=first_postcode
    except:
        print(postcode)
print("\nHouse price dictionary since 2022 has",len(hpp2022dict.keys()),"keys\n")

hpp2022dictdf=pd.DataFrame.from_dict(hpp2022dict,orient="index")
print("\nHouse price dictionary df (with split postcode) since 2022 has",hpp2022dictdf.shape[0],"rows")

hpp2022dictdf['Month of Transfer'] = pd.DatetimeIndex(hpp2022dictdf['Date of Transfer']).month
hpp2022dictdf['Month of Transfer']=hpp2022dictdf['Month of Transfer'].replace(1,"January")
hpp2022dictdf['Month of Transfer']=hpp2022dictdf['Month of Transfer'].replace(2,"Febuary")
hpp2022dictdf['Month of Transfer']=hpp2022dictdf['Month of Transfer'].replace(3,"March")
hpp2022dictdf['Month of Transfer']=hpp2022dictdf['Month of Transfer'].replace(4,"April")
hpp2022dictdf['Month of Transfer']=hpp2022dictdf['Month of Transfer'].replace(5,"May")

unique_sold_first_postcodes=list(hpp2022dictdf["First Postcode"].unique())

print("\nthere are",len(unique_sold_first_postcodes),"postcodes in UK that had a house sale in 2022\n")

singlesalepostcodes=hpp2022dictdf[hpp2022dictdf.groupby('First Postcode')['Price'].transform('size') == 1]
print("\nthere were",len(singlesalepostcodes["First Postcode"].unique()),"postcodes with a single sale in 2022\n")

hpp2022dictdf=hpp2022dictdf[hpp2022dictdf.groupby('First Postcode')['Price'].transform('size') > 1]
print("\nthere were",len(hpp2022dictdf["First Postcode"].unique()),"postcodes more than a single sale in 2022\n")

hpp2022dictdf["Price"]=hpp2022dictdf["Price"].astype(str)
hpp2022dictdfgrouped=hpp2022dictdf.groupby(['First Postcode'])['Price'].apply(','.join).reset_index()
hpp2022dictdfgrouped.head()
hpp2022dictdfgrouped.rename(columns={"Price":"All Prices for First Postcode"},inplace=True)
hpp2022dictdfgrouped.drop_duplicates(subset=["First Postcode"],inplace=True)
hpp2022dictdfgrouped.reset_index(inplace=True,drop=True)
print("house prices grouped by postcode has",hpp2022dictdfgrouped.shape[0],"rows")
#print("hpp2022dictdfgrouped=")
#hpp2022dictdfgrouped.head()


postcodes_with_train_times_df.head()
print("Train times for each postcode loaded and postcode split")

print("merging train time df with house prices grouped by first postcode...")
finaldf=pd.merge(hpp2022dictdfgrouped,postcodes_with_train_times_df[["First Postcode","train time to work"]],on="First Postcode")
print("Done")
finaldf.head(20)
finaldf["train time to work"]=finaldf["train time to work"].astype(str)
print("Joining finaldf train times into a single comma seperated string...")
commutedf=finaldf.groupby("First Postcode")['train time to work'].apply(','.join).reset_index()
commutedf.set_index("First Postcode",inplace=True)
print("Done")
print("merging finaldf with commuter df ...")
commutedf=pd.merge(finaldf,commutedf,on="First Postcode")
print("Done")
commutedf.drop(columns="train time to work_x",inplace=True)
commutedf.rename(columns={"train time to work_y":"train time to work"},inplace=True)
commutedf.drop_duplicates("First Postcode",inplace=True)
commutedf.head(20)
commutedf.set_index("First Postcode",inplace=True)
print("putting commutedf into a dictionary...")
commutedict=commutedf.to_dict(orient="index")
print("Done")
#print(commutedict)
hpp2022dictdfgrouped.head()

print("calculating the median prices and median times for each postcode")
for postcode,data in commutedict.items():
    data=commutedict.get(postcode)
    prices=data.get("All Prices for First Postcode").split(",")
    prices=[float(price) for price in prices]
    times=data.get("train time to work").split(",")
    times=[float(time) for time in times]
    mediantime=round(statistics.median(times))
    medianprice=round(statistics.median(prices))
    commutedict[postcode]['Median Price']=medianprice
    commutedict[postcode]['Median Time']=mediantime
print("Done")
print("putting the last df into a df to adjust index and drop columns")
lastdf=pd.DataFrame.from_dict(commutedict,orient="index")
lastdf.reset_index(inplace=True)
lastdf=lastdf[["index","Median Price","Median Time"]]
lastdf.set_index("index",inplace=True)
#print("lastdf")
lastdf.head(20)
print("Done")
print("putting the last df back into a dict as the last step")
lastdfdict=lastdf.to_dict(orient="index")
print("Lastdf written to dict")
print("lastdfdict = CF72")
lastdfdict.get("CF72")

# validated until here

postcodemappings.reset_index(inplace=True)
postcodemappings.set_index("index",inplace=True)
print("postcode df being written to df")
dict0=postcodemappings.to_dict(orient="index")
postcodeladict={}
print("postcode df dict being split to get first postcode")
for i in range(1,len(dict0.keys())):
    vals=list(dict0.get(i).values())
    #print(vals)
    PC=vals[1],"Local authority= ",vals[6],"first postcode=",vals[1].split(" ")[0]
    LAV=vals[6]
    FPC=vals[1].split(" ")[0]
    postcodeladict[FPC]=LAV
print("postcodeladict for CF72")
postcodeladict.get("CF72")

# OK UNTIL HERE

print("getting local authority for first postcode")
for postcode in list(lastdfdict.keys()):
    try:
        localauthority=postcodeladict[postcode]
        lastdfdict[postcode]["local authority"]=localauthority
    except:
        lastdfdict[postcode]["local authority"]="LA not known"
print("local authority in lastdfdict for CF72...")
lastdfdict["CF72"]

print("getting population for local authority")

for postcode in list(lastdfdict.keys()):
    try:
        localauthority=lastdfdict.get(postcode)["local authority"]
        population=list(population_infodict[localauthority].values())[0]
        lastdfdict[postcode]["Population"]=population
    except:
        lastdfdict[postcode]["Population"]="Population not known"
print("lastdfdict for CF72 including population")
lastdfdict["CF72"]


print("getting deprivation rank for local authority")

deprivation_scores_grouped=deprivation_scores.groupby('Local Authority Name')[["WIMD "]].median()
deprivation_scores_grouped.loc["ISLE OF ANGLESEY"][0]
alldepscores=[]
for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        deprivationscore=deprivation_scores_grouped.loc[localauthority][0]
        alldepscores.append(deprivationscore)
        lastdfdict[postcode]["Deprivation Rank"]=deprivationscore
    except:
        lastdfdict[postcode]["Deprivation Rank"]="Deprivation Unknown"
lastdfdict["CF72"]





# OK UNTIL HERE #






print("getting median price for area relative to national median price")

allprices=[]
for postcode in list(lastdfdict.keys()):
    allprices.append(lastdfdict[postcode]["Median Price"])
allmedianprice=statistics.median(allprices)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Postcode Median Price Relative to National Median Price"]=lastdfdict[postcode]["Median Price"]/allmedianprice
lastdfdict["CF72"]

print("getting median time for area relative to national median time")

alltimes=[]
for postcode in list(lastdfdict.keys()):
    alltimes.append(lastdfdict[postcode]["Median Time"])
allmediantime=statistics.median(alltimes)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Postcode Median Time Relative to National Median Time"]=lastdfdict[postcode]["Median Time"]/allmediantime
lastdfdict["CF72"]


print("getting retail counts for each local authority")

for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        retaildict=retail_infodict.get(localauthority)
        retailcount=list(retaildict.values())[0]
        lastdfdict[postcode]["Retail Count"]=retailcount
    except:
        lastdfdict[postcode]["Retail Count"]="Retail Count Unknown"
lastdfdict["CF72"]



#print("getting crime counts for local authority")
#
#for postcode in list(lastdfdict.keys()):
#    localauthority=lastdfdict[postcode]["local authority"]
#    try:
#        crimecount=list(crimedfdict.get(localauthority).values())[0]
#        lastdfdict[postcode]["Crime Count"]=crimecount
#    except:
#        lastdfdict[postcode]["Crime Count"]="Crime Count Unknown"
#lastdfdict["CF72"]


print("getting area for local authority")

for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        area=list(areasizeinfodict.get(localauthority).values())[0]
        lastdfdict[postcode]["Area"]=area
    except:
        lastdfdict[postcode]["Area"]="Area Unknown"
lastdfdict["CF72"]

print("area calculations done")


maxbeachcount=max(list(beachcountdict.values()))

postcodeswithoutbeaches=[]

print("getting beach count for first postcodes")

for postcode in list(lastdfdict.keys()):
    try:
        postcodestem=re.sub(r'\d+', '', postcode)
        beachcount=beachcountbypostcodestem[postcodestem]
        lastdfdict[postcode]["Beach Count"]=beachcount
        lastdfdict[postcode]["Beach Score (Relative to Max)"]=beachcount/maxbeachcount
    except:
        postcodeswithoutbeaches.append(postcode)
        lastdfdict[postcode]["Beach Count"]=0
        lastdfdict[postcode]["Beach Score (Relative to Max)"]=0
lastdfdict["CF72"]

print("Beach count done")

#OK UNTIL HERE




print("getting postcodes with any unknown values")


unknowndict={}
for postcode in list(lastdfdict.keys()):
    lanacount = 0
    if (lastdfdict[postcode]["local authority"]=="LA not known"):
        lanacount += 1
    popnacount = 0
    if (lastdfdict[postcode]["Population"]=="Population not known"):
        popnacount += 1
    dinacount = 0
    if (lastdfdict[postcode]["Deprivation Rank"]=="Deprivation Unknown"):
        dinacount += 1
    areanacount=0
    if (lastdfdict[postcode]["Area"]=="Area Unknown"):
        areanacount += 1
#    ccnacount = 0
#    if (lastdfdict[postcode]["Crime Count"]=="Crime Count Unknown"):
#        ccnacount += 1
    retailnacount = 0
    if (lastdfdict[postcode]["Retail Count"]=="Retail Count Unknown"):
        retailnacount += 1
    if sum([lanacount,popnacount,dinacount,areanacount,retailnacount])>0:
        unknowndict[postcode]={"LA Unknown":lanacount,"Pop Unknown":popnacount,"DepInd Unknown":dinacount,"Area Unknown":areanacount,"Retail Unknown":retailnacount}
postcodes_to_drop=list(unknowndict.keys())
print(len(postcodes_to_drop))
print("dropping any postcodes from lastdfdict with any NA values")
print(len(lastdfdict))
for postcode in list(lastdfdict.keys()):
    if postcode in postcodes_to_drop:
        del lastdfdict[postcode]
print(len(lastdfdict))


#Deprivation Rank compared to mean 

mediandepscore=statistics.median(alldepscores)
print(mediandepscore)
for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    depscore=deprivation_scores_grouped.loc[localauthority][0]
    relativedepscore=depscore/mediandepscore
    lastdfdict[postcode]["Deprivation Score Relative to National Median"]=relativedepscore
lastdfdict["CF72"]

popdensitylist=[]
for postcode in list(lastdfdict.keys()):
    popdensity=lastdfdict[postcode]["Population"]/lastdfdict[postcode]["Area"]
    lastdfdict[postcode]["Population Density"]=popdensity
    popdensitylist.append(popdensity)

medianpopdensity=statistics.median(popdensitylist)    
for postcode in list(lastdfdict.keys()):
    relativepopdensity=lastdfdict[postcode]["Population Density"]/medianpopdensity
    lastdfdict[postcode]["Population Density (Relative to Median)"]=relativepopdensity

# Total crimes per 1000 pop

#for postcode in list(lastdfdict.keys()):
#    relativecrimescore=lastdfdict[postcode]['Crime Count']/lastdfdict[postcode]['Population']
#    lastdfdict[postcode]["Crimes per 1000"]=relativecrimescore
#lastdfdict["E10"]

# LocAuth Crime rate compared to median Crime rate

#allcrimerates=[]
#for postcode in list(lastdfdict.keys()):
#    allcrimerates.append(lastdfdict[postcode]["Crimes per 1000"])
#allmediancrimes=statistics.median(allcrimerates)
#for postcode in list(lastdfdict.keys()):
#    lastdfdict[postcode]["Crime rate relative to National Median"]=lastdfdict[postcode]["Crimes per 1000"]/allmediancrimes
#lastdfdict["E10"]


#Retail Businesses per Hectare

for postcode in list(lastdfdict.keys()):
    retailperhectare=lastdfdict[postcode]['Retail Count']/lastdfdict[postcode]['Area']
    lastdfdict[postcode]["Businesses per Hectere"]=retailperhectare
lastdfdict["CF72"]

# LocAuth retail density compared to Median retail density

allretaildensity=[]
for postcode in list(lastdfdict.keys()):
    allretaildensity.append(lastdfdict[postcode]["Businesses per Hectere"])
allmedianretaildensity=statistics.median(allretaildensity)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Retail Score Relative to National Median"]=lastdfdict[postcode]["Businesses per Hectere"]/allmedianretaildensity
lastdfdict["CF72"]
print("all done")

shape of input df is (299432, 16)

this is the breakdown by house type across all UK
 Terraced         93233
Semi-Detached    82264
Detached         61461
Flat             52139
Other            10335
Name: Property Type, dtype: int64

 houses sold since 2022
 (52139, 16)


C:\Users\Carl\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


hpp2022dict[2]= {'Transaction unique identifier': '{E53EDD2D-A31C-83EC-E053-6B04A8C03A59}', 'Price': 135000, 'Date of Transfer': '2022-06-22 00:00', 'Postcode': 'CM0 7EG', 'Property Type': 'Flat', 'Old/New': 'N', 'Duration': 'L', 'PAON': '45', 'SAON': nan, 'Street': 'TATTERSALLS CHASE', 'Locality': nan, 'Town/City': 'SOUTHMINSTER', 'District': 'MALDON', 'County': 'ESSEX', 'PPD Category Type': 'A', 'TBD': 'A'}

House price dictionary since 2022 has 52124 keys


House price dictionary df (with split postcode) since 2022 has 52124 rows

there are 2093 postcodes in UK that had a house sale in 2022


there were 140 postcodes with a single sale in 2022


there were 1953 postcodes more than a single sale in 2022

house prices grouped by postcode has 1953 rows
Train times for each postcode loaded and postcode split
merging train time df with house prices grouped by first postcode...
Done
Joining finaldf train times into a single comma seperated string...
Done
merging finaldf with commuter df .

In [30]:
#lastdfdict={key:val for key,val in lastdfdict.items() if 400000<lastdfdict[key]["Median Price"]<550000}

In [32]:
#lastdfdict.keys()

dict_keys(['BN12', 'BN14', 'BN18', 'BN2', 'BN42', 'BN43', 'BN8', 'BR1', 'BR5', 'CB1', 'CB10', 'CB11', 'CB22', 'CB4', 'CM1', 'CM12', 'CM13', 'CM14', 'CM21', 'CM22', 'CM23', 'CM24', 'CM3', 'CM77', 'CO5', 'CR0', 'CR2', 'CR3', 'CR4', 'CR7', 'CT10', 'CT4', 'CT5', 'DA15', 'DA16', 'DA5', 'DA7', 'E12', 'E14', 'E15', 'E4', 'E7', 'EN1', 'EN10', 'EN11', 'EN3', 'EN8', 'GL7', 'GU16', 'GU21', 'GU32', 'GU33', 'GU34', 'GU47', 'HA2', 'HA3', 'HP23', 'HP3', 'IG1', 'KT14', 'KT15', 'KT16', 'KT9', 'ME19', 'MK17', 'MK45', 'N17', 'N9', 'OX1', 'OX29', 'OX7', 'PO19', 'RG41', 'RG6', 'RG7', 'RG8', 'RH1', 'RH12', 'RH13', 'RH14', 'RH15', 'RH16', 'RH19', 'RH4', 'RH6', 'RH7', 'RM1', 'RM6', 'SE12', 'SE18', 'SE2', 'SE25', 'SE6', 'SE9', 'SG12', 'SG13', 'SG6', 'SG7', 'SG8', 'SL1', 'SM1', 'SM4', 'SM5', 'SM6', 'SN8', 'SN9', 'SO23', 'SS1', 'SS6', 'SS7', 'SS9', 'TN15', 'TN19', 'TN2', 'TN25', 'TN26', 'TN27', 'TN31', 'TN33', 'TN36', 'TN6', 'TN8', 'TN9', 'TW13', 'TW15', 'TW16', 'TW17', 'TW18', 'TW19', 'TW20', 'TW3', 'UB2', 'UB3

In [17]:
maxmedprice=0
for postcode in list(lastdfdict.keys()):
    medprice=lastdfdict[postcode]["Median Price"]
    if medprice>maxmedprice:
        maxmedprice=medprice
print("maxprice",maxmedprice)
maxbusinesses=0
for postcode in list(lastdfdict.keys()):
    businessdens=lastdfdict[postcode]["Businesses per Hectere"]
    if businessdens>maxbusinesses:
        maxbusinesses=businessdens
print("maxbusinesses",maxbusinesses)
maxdeprank=0
for postcode in list(lastdfdict.keys()):
    deprank=lastdfdict[postcode]["Deprivation Rank"]
    if deprank>maxdeprank:
        maxdeprank=deprank
print("maxdep",maxdeprank)
#maxcrimerank=0
#for postcode in list(lastdfdict.keys()):
#    crimerank=lastdfdict[postcode]["Crimes per 1000"]
#    if crimerank>maxcrimerank:
#        maxcrimerank=crimerank
#print("maxcrime",maxcrimerank)
maxtimerank=0
for postcode in list(lastdfdict.keys()):
    timerank=lastdfdict[postcode]["Median Time"]
    if timerank>maxtimerank:
        maxtimerank=timerank
print("maxtime",maxtimerank)


maxpopdens=0
for postcode in list(lastdfdict.keys()):
    popdens=lastdfdict[postcode]["Population Density"]
    if popdens>maxpopdens:
        maxpopdens=popdens
print("maxpopdens",maxpopdens)


maxprice 315000
maxbusinesses 0.06323400798555187
maxdep 1553.0
maxtime 689
maxpopdens 24.249740205252905


In [35]:
lastdfdict["CF72"]

{'Median Price': 220000,
 'Median Time': 500,
 'local authority': 'RHONDDA CYNON TAF',
 'Population': 237700,
 'Deprivation Rank': 624.0,
 'Postcode Median Price Relative to National Median Price': 1.3333333333333333,
 'Postcode Median Time Relative to National Median Time': 3.4722222222222223,
 'Retail Count': 685.0,
 'Area': 42415.02,
 'Beach Count': 8,
 'Beach Score (Relative to Max)': 0.7272727272727273,
 'Deprivation Score Relative to National Median': 0.5948522402287894,
 'Population Density': 5.604146832890802,
 'Population Density (Relative to Median)': 1.7796037187429825,
 'Businesses per Hectere': 0.016149939337527133,
 'Retail Score Relative to National Median': 2.7538718106116256}

In [18]:
lastdfdict["CF72"]

{'Median Price': 220000,
 'Median Time': 500,
 'local authority': 'RHONDDA CYNON TAF',
 'Population': 237700,
 'Deprivation Rank': 624.0,
 'Postcode Median Price Relative to National Median Price': 1.3333333333333333,
 'Postcode Median Time Relative to National Median Time': 3.4722222222222223,
 'Retail Count': 685.0,
 'Area': 42415.02,
 'Beach Count': 8,
 'Beach Score (Relative to Max)': 0.7272727272727273,
 'Deprivation Score Relative to National Median': 0.5948522402287894,
 'Population Density': 5.604146832890802,
 'Population Density (Relative to Median)': 1.7796037187429825,
 'Businesses per Hectere': 0.016149939337527133,
 'Retail Score Relative to National Median': 2.7538718106116256}

In [24]:
welshgreendf.index.name="local authority"
welshgreendf.head()

,Median distance to nearest park or public garden (m),Median size of nearest park or public garden (m2),"Median number of parks and public gardens in 1,000 m radius","Median combined size of parks and public gardens in 1,000 m radius (m2)",Median population per park or public garden,Number of flats in rural and built up areas,Number of houses in rural and built up area,Median distance to nearest park or public garden (m) (Relative to Max),Median size of nearest park or public garden (m2) (Relative to Max),"Median number of parks and public gardens in 1,000 m radius (Relative to Max)","Median combined size of parks and public gardens in 1,000 m radius (m2) (Relative to Max)",Median population per park or public garden (Relative to Max),Number of flats in rural and built up areas (Relative to Max),Number of houses in rural and built up area (Relative to Max),Green Score (Raw),Green Score (Relative to Max)
local authority,,,,,,,,,,,,,,,,
CARDIFF,366.863705,23055.81460,3,198447,9740.948555,47617,109346,0.082419,0.277304,1.000000,0.996575,0.617003,1.000000,1.000000,19.597159,1.000000
RHONDDA CYNON TAF,951.365770,41231.53730,1,58708,14255.024178,12461,98424,0.213734,0.495913,0.333333,0.294824,0.902929,0.261692,0.900115,0.509231,0.025985
SWANSEA,567.225700,35063.82660,1,62760,11741.340807,22112,91916,0.127433,0.421731,0.333333,0.315173,0.743710,0.464372,0.840598,1.108518,0.056565
CARMARTHENSHIRE,2625.026100,33666.49795,1,40465,10802.059058,9243,78969,0.589738,0.404924,0.333333,0.203210,0.684214,0.194111,0.722194,0.167870,0.008566
CAERPHILLY,695.307820,46088.47240,1,67095,9770.802994,7829,71927,0.156208,0.554330,0.333333,0.336942,0.618894,0.164416,0.657793,1.161759,0.059282


In [25]:
for postcode in list(lastdfdict.keys()):
    price_score=(lastdfdict[postcode]['Median Price']/maxmedprice)**2
    lastdfdict[postcode]["Price Score (Relative to Max)"]=price_score
    # smaller is better
    retail_score=(lastdfdict[postcode]["Businesses per Hectere"]/maxbusinesses)
    lastdfdict[postcode]["Retail Score (Relative to Max)"]=retail_score
    # bigger is better
    deprivation_score=(lastdfdict[postcode]["Deprivation Rank"]/maxdeprank)**2
    lastdfdict[postcode]["Deprivation Score (Relative to Max)"]=deprivation_score
    # bigger is better
    #crime_score=(lastdfdict[postcode]["Crimes per 1000"]/maxcrimerank)**1.25
    # smaller is better
    time_score=(lastdfdict[postcode]["Median Time"]/maxtimerank)
    lastdfdict[postcode]["Time Score (Relative to Max)"]=time_score
    
    #smaller is better
    population_density_score=(lastdfdict[postcode]["Population Density"]/maxpopdens)
    lastdfdict[postcode]["Population Density (Relative to Max)"]=population_density_score
    
    beach_score=lastdfdict[postcode]["Beach Score (Relative to Max)"]
    
    # smaller is better
    #lastdfdict[postcode]["Total Score (Postcode)"]=(deprivation_score*retail_score*beach_score)/(price_score)
    # bigger is better
outputdf=pd.DataFrame.from_dict(data=lastdfdict,orient="index")
outputdf=outputdf[["local authority","Median Price","Price Score (Relative to Max)","Median Time","Time Score (Relative to Max)","Retail Count","Population","Area","Population Density","Businesses per Hectere","Retail Score (Relative to Max)","Deprivation Rank","Deprivation Score (Relative to Max)","Beach Count","Beach Score (Relative to Max)"]]
finaldf=pd.merge(outputdf.reset_index().set_index("local authority"),welshgreendf,on="local authority")
finaldf=finaldf.rename(columns={"index":"Postcode"})
finaldf=finaldf.reset_index()
finaldf=finaldf.set_index("Postcode")
finaldf["Total Score (Postcode)"]=(finaldf["Deprivation Score (Relative to Max)"])/(finaldf["Price Score (Relative to Max)"])
savedf=finaldf.sort_values(by="Total Score (Postcode)",ascending=False)
savedf.to_excel("flats_within_wales.xlsx")

In [68]:
finaldf["Total Score (Postcode)"]

Postcode
CF10    4.466472
CF11    4.677144
CF14    4.531841
CF15    4.654661
CF23    5.291695
CF5     4.900821
CF31    0.016964
CF32    0.011104
CF33    0.009812
CF37    0.031265
CF38    0.025013
CF44    0.039232
CF45    0.030310
CF72    0.002201
CF47    0.033852
CF48    0.032769
CF62    0.280216
CF63    0.286677
CF64    0.165136
CF81    0.037482
CF82    0.034406
CF83    0.030239
NP11    0.000000
CH4     0.000000
CH5     0.000000
CH7     0.000000
LL11    0.076775
LL12    0.052957
LL18    0.148000
LD1     0.000000
LD2     0.000000
LD5     0.000000
SY16    0.000695
LL13    0.667768
LL14    1.102158
LL19    0.103695
LL28    0.003246
LL29    0.019121
LL30    0.004751
LL31    0.004394
LL33    0.002147
LL34    0.003509
LL35    0.000137
LL42    0.000276
LL46    0.001307
LL49    0.000282
LL52    0.000185
LL53    0.000449
LL57    0.000813
LL64    0.000039
LL65    0.000094
NP10    0.000000
NP20    0.000000
NP13    0.000000
NP23    0.000000
NP16    0.000000
NP26    0.000000
NP7     0.000000
NP4  

In [ ]:
savedf=outputdf.sort_values(by="Total Score (Postcode)",ascending=False)
savedf.to_excel("flats_within_wales.xlsx")

In [6]:
os.getcwd()

'C:\\Users\\Carl\\Housing'

# Flat calculations

In [7]:
columns=["Transaction unique identifier","Price","Date of Transfer","Postcode","Property Type","Old/New","Duration","PAON","SAON","Street","Locality","Town/City","District","County","PPD Category Type","TBD"]
postcodes_with_train_times_df=pd.read_excel("C:\\Users\\Carl\\Housing\\postcodedfwithtimedf.xlsx")
pp2022=pd.read_csv("C:\\Users\\Carl\\Housing\\Data\\pp-2022.csv")
pp2022.columns=columns
postcodemappings=pd.read_csv(open("C:\\Users\\Carl\\Housing\\Data\\Postcode to LA.csv"),sep='\t',error_bad_lines=False)

deprivation_scores=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\Deprivation Scores.xlsx','rb'),"IMD2019")

retail_info=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\RetailInfo.xlsx','rb'),"Table 1",usecols=[1,7],skiprows=range(5))
retail_info=retail_info.dropna()
retail_info=retail_info.rename(columns={"Unnamed: 1":"Local Authority","47 : Retail":"Count"})
retail_info.set_index("Local Authority",inplace=True)
retail_infodict=retail_info.to_dict(orient="index")

crimedf=pd.read_excel(open("C:\\Users\\Carl\\Housing\\Data\\Crime Data.xlsx","rb"),"Table C3",skiprows=range(6))
crimedf.columns=[string.replace("\n","") for string in list(crimedf.columns)]
crimedf=crimedf[["Local Authority name","Total recorded crime (excluding fraud)"]]
crimedf.dropna(subset=["Local Authority name"],inplace=True)
crimedf=crimedf.set_index("Local Authority name")
crimedfdict=crimedf.to_dict(orient="index")

population_info=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\PopulationFigures.xlsx','rb'),"Ethnicity",usecols=[3,4],skiprows=range(6))
population_info=population_info.dropna()
population_info=population_info.rename(columns={" .1":"Local Authority","Number":"Population"})
population_info.set_index("Local Authority",inplace=True)
population_infodict=population_info.to_dict(orient="index")

areasizeinfo=pd.read_excel(open('C:\\Users\\Carl\\Housing\\Data\\SAM_LAD_DEC_2020_UK.xlsx','rb'),"SAM_LAD_2020",usecols=[1,2])
areasizeinfo=areasizeinfo.dropna()
areasizeinfo.rename(columns={"LAD20NM":"Local Authority","AREAEHECT":"Area (Hectares)"},inplace=True)
areasizeinfo.set_index("Local Authority",inplace=True)
areasizeinfodict=areasizeinfo.to_dict(orient="index")
areasizeinfodict["Hartlepool"]

dct={}
for year in range(2008,2021):
    df=pd.read_csv(str(os.getcwd())+"\\Data\\square footage\\KT1_"+str(year)+"_flat_data.csv",usecols=[1,2,3,4,31])
    dct[year]=df
flatsizesinkingston=pd.concat(dct.values())
flatsizesinkingston=flatsizesinkingston.reset_index()
flatsizesinkingston=flatsizesinkingston.drop(columns=["index"])
medianflatsizeinkingstonla=flatsizesinkingston[["total-floor-area"]].median().values[0]
print("medianflatsizeinkingstonla=",medianflatsizeinkingstonla)

medianflatsizeinkingstonla= 58.0


In [8]:
pd.set_option('display.max_rows', None)
#pp2020=pd.read_csv("C:\\Users\\Carl\\Housing\\Data\\pp-2020.csv")
#pp2020.columns=columns
#pp2021=pd.read_csv("C:\\Users\\Carl\\Housing\\Data\\pp-2021.csv")
#pp2021.columns=columns

print("shape of input df is",pp2022.shape)

pp2022['Property Type']=pp2022["Property Type"].str.replace("D","Detached")
pp2022['Property Type']=pp2022["Property Type"].str.replace("S","Semi-Detached")
pp2022['Property Type']=pp2022["Property Type"].str.replace("T","Terraced")
pp2022['Property Type']=pp2022["Property Type"].str.replace("F","Flat")
pp2022["Property Type"]=pp2022["Property Type"].str.replace("O","Other")

print("\nthis is the breakdown by house type across all areas\n",pp2022["Property Type"].value_counts())

fpp2022=pp2022[pp2022["Property Type"]=="Flat"]
print("\n flats sold since 2022\n",fpp2022.shape)

fpp2022.reset_index(inplace=True,drop=True)

fpp2022.dropna(subset=["Postcode"],inplace=True)
fpp2022.reset_index(inplace=True,drop=True)

fpp2022dict=fpp2022.to_dict(orient="index")
print(fpp2022dict[2])

for i in range(0,len(fpp2022)):
    try:
        postcode=list(fpp2022dict.get(i).items())[3][1]
        first_postcode=postcode.split(" ")[0]
        fpp2022dict[i]["First Postcode"]=first_postcode
    except:
        print(postcode)
print("\nFlat price dictionary since 2022 has",len(fpp2022dict.keys()),"keys\n")

fpp2022dictdf=pd.DataFrame.from_dict(fpp2022dict,orient="index")
print("\nFlat price dictionary df (with split postcode) since 2022 has",fpp2022dictdf.shape[0],"rows")


fpp2022dictdf['Month of Transfer'] = pd.DatetimeIndex(fpp2022dictdf['Date of Transfer']).month
fpp2022dictdf['Month of Transfer']=fpp2022dictdf['Month of Transfer'].replace(1,"January")
fpp2022dictdf['Month of Transfer']=fpp2022dictdf['Month of Transfer'].replace(2,"Febuary")
fpp2022dictdf['Month of Transfer']=fpp2022dictdf['Month of Transfer'].replace(3,"March")
fpp2022dictdf['Month of Transfer']=fpp2022dictdf['Month of Transfer'].replace(4,"April")
fpp2022dictdf['Month of Transfer']=fpp2022dictdf['Month of Transfer'].replace(5,"May")

unique_sold_first_postcodes=list(fpp2022dictdf["First Postcode"].unique())

print("\nthere are",len(unique_sold_first_postcodes),"postcodes that had a flat sale in 2022\n")

singlesalepostcodes=fpp2022dictdf[fpp2022dictdf.groupby('First Postcode')['Price'].transform('size') == 1]
print("\nthere were",len(singlesalepostcodes["First Postcode"].unique()),"postcodes with a single sale in 2022\n")

fpp2022dictdf=fpp2022dictdf[fpp2022dictdf.groupby('First Postcode')['Price'].transform('size') > 1]
print("\nthere were",len(fpp2022dictdf["First Postcode"].unique()),"postcodes more than a single sale in 2022\n")

fpp2022dictdf["Price"]=fpp2022dictdf["Price"].astype(str)
fpp2022dictdfgrouped=fpp2022dictdf.groupby(['First Postcode'])['Price'].apply(','.join).reset_index()
fpp2022dictdfgrouped
fpp2022dictdfgrouped.rename(columns={"Price":"All Prices for First Postcode"},inplace=True)
fpp2022dictdfgrouped.drop_duplicates(subset=["First Postcode"],inplace=True)
fpp2022dictdfgrouped.reset_index(inplace=True,drop=True)
print("flat prices grouped by postcode has",fpp2022dictdfgrouped.shape[0],"rows")

fpp2022dictdfgrouped[fpp2022dictdfgrouped["First Postcode"]=="KT1"]["All Prices for First Postcode"].values

postcodes_with_train_times_df.drop(columns=["Unnamed: 0"],inplace=True)
print("splitting postcodes in train time df...")
postcodes_with_train_times_df["First Postcode"]="tbd"
for i in range(0,len(postcodes_with_train_times_df)):
    postcode=postcodes_with_train_times_df.loc[i]["postcode"]
    split_postcode=postcode.split(" ")
    postcodes_with_train_times_df.loc[postcodes_with_train_times_df['postcode'] == postcode, 'First Postcode'] = split_postcode[0]
postcodes_with_train_times_df.head()
#postcodes_with_train_times_df=postcodes_with_train_times_df[postcodes_with_train_times_df["train time to work"]<75]
print("train times all gathered...")
print("merging train time df with house prices grouped by first postcode...")
finaldf=pd.merge(fpp2022dictdfgrouped,postcodes_with_train_times_df[["First Postcode","train time to work"]],on="First Postcode")
print("Done")
finaldf["train time to work"]=finaldf["train time to work"].astype(str)
print("Joining finaldf train times into a single comma seperated string...")
commutedf=finaldf.groupby("First Postcode")['train time to work'].apply(','.join).reset_index()
#commutedf.set_index("First Postcode",inplace=True)
print("Done")
print("merging finaldf with commuter df ...")
commutedf=pd.merge(finaldf,commutedf,on="First Postcode")
print("Done")
commutedf.drop(columns="train time to work_x",inplace=True)
commutedf.rename(columns={"train time to work_y":"train time to work"},inplace=True)
commutedf.drop_duplicates("First Postcode",inplace=True)
commutedf.head()
commutedf.set_index("First Postcode",inplace=True)
print("putting commutedf into a dictionary...")
commutedict=commutedf.to_dict(orient="index")
print("Done")
#print(commutedict)
#hpp2022dictdfgrouped.head()

shape of input df is (105501, 16)

this is the breakdown by house type across all areas
 Terraced         32859
Semi-Detached    28638
Detached         21687
Flat             18063
Other             4254
Name: Property Type, dtype: int64

 flats sold since 2022
 (18063, 16)


C:\Users\Carl\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'Transaction unique identifier': '{DE2D0CDF-7F05-51EE-E053-6C04A8C00671}', 'Price': 370000, 'Date of Transfer': '2022-03-31 00:00', 'Postcode': 'EX8 1BG', 'Property Type': 'Flat', 'Old/New': 'N', 'Duration': 'L', 'PAON': '17', 'SAON': 'FLAT 1', 'Street': 'MORTON CRESCENT', 'Locality': nan, 'Town/City': 'EXMOUTH', 'District': 'EAST DEVON', 'County': 'DEVON', 'PPD Category Type': 'A', 'TBD': 'A'}

Flat price dictionary since 2022 has 18056 keys


Flat price dictionary df (with split postcode) since 2022 has 18056 rows

there are 1885 postcodes that had a flat sale in 2022


there were 245 postcodes with a single sale in 2022


there were 1640 postcodes more than a single sale in 2022

flat prices grouped by postcode has 1640 rows
splitting postcodes in train time df...
train times all gathered...
merging train time df with house prices grouped by first postcode...
Done
Joining finaldf train times into a single comma seperated string...
Done
merging finaldf with commuter df ...
Done
putt

In [10]:
print("calculating the median prices and median times for each postcode")
for postcode,data in commutedict.items():
    data=commutedict.get(postcode)
    prices=data.get("All Prices for First Postcode").split(",")
    prices=[float(price) for price in prices]
    times=data.get("train time to work").split(",")
    times=[float(time) for time in times]
    mediantime=round(statistics.median(times))
    medianprice=round(statistics.median(prices))
    if postcode=="KT1":
        print(medianprice,postcode,"this is the median price for KT1")
    commutedict[postcode]['Median Price']=medianprice
    commutedict[postcode]['Median Time']=mediantime
print("Done")
print("putting the last df into a df to adjust index and drop columns")
lastdf=pd.DataFrame.from_dict(commutedict,orient="index")
lastdf.reset_index(inplace=True)
lastdf=lastdf[["index","Median Price","Median Time"]]
lastdf.set_index("index",inplace=True)
print("Done")
print("putting the last df back into a dict as the last step")
lastdfdict=lastdf.to_dict(orient="index")
print("Lastdf written to dict")
lastdfdict.get("E10")

postcodemappings.reset_index(inplace=True)
postcodemappings.set_index("index",inplace=True)
print("postcode df being written to df")
dict0=postcodemappings.to_dict(orient="index")
postcodeladict={}
print("postcode df dict being split to get first postcode")
for i in range(1,len(dict0.keys())):
    vals=list(dict0.get(i).values())
    for string in vals:
        PC=string.split(",")[1]
        LAV=string.split(",")[6]
        FPC=PC.split(" ")[0]
    postcodeladict[FPC]=LAV
postcodeladict.get("E10")
print("getting local authority for first postcode")
for postcode in list(lastdfdict.keys()):
    try:
        localauthority=postcodeladict[postcode]
        lastdfdict[postcode]["local authority"]=localauthority
    except:
        lastdfdict[postcode]["local authority"]="LA not known"
lastdfdict["E10"]

for postcode in list(lastdfdict.keys()):
    if "Kingston upon Thames" in lastdfdict[postcode]["local authority"]:
        lastdfdict[postcode]['Median Price over Median Area (Kingston Only)']=405000/medianflatsizeinkingstonla
        print(405000/medianflatsizeinkingstonla)
    else:
        lastdfdict[postcode]['Median Price over Median Area (Kingston Only)']=0


print("getting population for local authority")

for postcode in list(lastdfdict.keys()):
    try:
        localauthority=lastdfdict.get(postcode)["local authority"]
        population=list(population_infodict[localauthority].values())[0]
        lastdfdict[postcode]["Population"]=population
    except:
        lastdfdict[postcode]["Population"]="Population not known"
lastdfdict["E10"]


print("getting deprivation rank for local authority")

deprivation_scores_grouped=deprivation_scores.groupby('Local Authority District name (2019)')[["Index of Multiple Deprivation (IMD) Rank"]].median()
deprivation_scores_grouped.loc["Adur"][0]
alldepscores=[]
for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        deprivationscore=deprivation_scores_grouped.loc[localauthority][0]
        alldepscores.append(deprivationscore)
        lastdfdict[postcode]["Deprivation Rank"]=deprivationscore
    except:
        lastdfdict[postcode]["Deprivation Rank"]="Deprivation Unknown"
lastdfdict["E10"]


print("getting median price for area relative to median price within 2 hrs of work")

allprices=[]
for postcode in list(lastdfdict.keys()):
    allprices.append(lastdfdict[postcode]["Median Price"])
allmedianprice=statistics.median(allprices)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Postcode Median Price Relative to 2hr Radius Median Price"]=lastdfdict[postcode]["Median Price"]/allmedianprice
lastdfdict["E10"]


alltimes=[]
for postcode in list(lastdfdict.keys()):
    alltimes.append(lastdfdict[postcode]["Median Time"])
allmediantime=statistics.median(alltimes)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Postcode Median Time Relative to 2hr Radius Median Time"]=lastdfdict[postcode]["Median Time"]/allmediantime
lastdfdict["E10"]


print("getting retail counts for each local authority")

for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        retaildict=retail_infodict.get(localauthority)
        retailcount=list(retaildict.values())[0]
        lastdfdict[postcode]["Retail Count"]=retailcount
    except:
        lastdfdict[postcode]["Retail Count"]="Retail Count Unknown"
lastdfdict["E10"]


print("getting crime counts for local authority")

for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        crimecount=list(crimedfdict.get(localauthority).values())[0]
        lastdfdict[postcode]["Crime Count"]=crimecount
    except:
        lastdfdict[postcode]["Crime Count"]="Crime Count Unknown"
lastdfdict["E10"]

print("getting area for local authority")

for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    try:
        area=list(areasizeinfodict.get(localauthority).values())[0]
        lastdfdict[postcode]["Area"]=area
    except:
        lastdfdict[postcode]["Area"]="Area Unknown"
lastdfdict["E10"]





print("getting postcodes with any unknown values")


unknowndict={}
for postcode in list(lastdfdict.keys()):
    lanacount = 0
    if (lastdfdict[postcode]["local authority"]=="LA not known"):
        lanacount += 1
    popnacount = 0
    if (lastdfdict[postcode]["Population"]=="Population not known"):
        popnacount += 1
    dinacount = 0
    if (lastdfdict[postcode]["Deprivation Rank"]=="Deprivation Unknown"):
        dinacount += 1
    areanacount=0
    if (lastdfdict[postcode]["Area"]=="Area Unknown"):
        areanacount += 1
    ccnacount = 0
    if (lastdfdict[postcode]["Crime Count"]=="Crime Count Unknown"):
        ccnacount += 1
    retailnacount = 0
    if (lastdfdict[postcode]["Retail Count"]=="Retail Count Unknown"):
        retailnacount += 1
    if sum([lanacount,popnacount,dinacount,areanacount,ccnacount,retailnacount])>0:
        unknowndict[postcode]={"LA Unknown":lanacount,"Pop Unknown":popnacount,"DepInd Unknown":dinacount,"Area Unknown":areanacount,"CC Unknown":ccnacount,"Retail Unknown":retailnacount}
postcodes_to_drop=list(unknowndict.keys())

print("dropping any postcodes from lastdfdict with any NA values")

for postcode in list(lastdfdict.keys()):
    if postcode in postcodes_to_drop:
        del lastdfdict[postcode]
print(len(lastdfdict))


# Deprivation Rank compared to mean 

mediandepscore=statistics.median(alldepscores)
print(mediandepscore)
for postcode in list(lastdfdict.keys()):
    localauthority=lastdfdict[postcode]["local authority"]
    #print(localauthority)
    depscore=deprivation_scores_grouped.loc[localauthority][0]
    relativedepscore=depscore/mediandepscore
    lastdfdict[postcode]["Deprivation Score Relative to Nation Median"]=relativedepscore
lastdfdict["E10"]

# Total crimes per 1000 pop

for postcode in list(lastdfdict.keys()):
    relativecrimescore=lastdfdict[postcode]['Crime Count']/lastdfdict[postcode]['Population']
    lastdfdict[postcode]["Crimes per 1000"]=relativecrimescore
lastdfdict["E10"]

# LocAuth Crime rate compared to median Crime rate

allcrimerates=[]
for postcode in list(lastdfdict.keys()):
    allcrimerates.append(lastdfdict[postcode]["Crimes per 1000"])
allmediancrimes=statistics.median(allcrimerates)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Crime rate relative to National Median"]=lastdfdict[postcode]["Crimes per 1000"]/allmediancrimes
lastdfdict["E10"]


# Retail Businesses per Hectare

for postcode in list(lastdfdict.keys()):
    retailperhectare=lastdfdict[postcode]['Retail Count']/lastdfdict[postcode]['Area']
    lastdfdict[postcode]["Businesses per Hectere"]=retailperhectare
lastdfdict["E10"]

# LocAuth retail density compared to Median retail density

allretaildensity=[]
for postcode in list(lastdfdict.keys()):
    allretaildensity.append(lastdfdict[postcode]["Businesses per Hectere"])
allmedianretaildensity=statistics.median(allretaildensity)
for postcode in list(lastdfdict.keys()):
    lastdfdict[postcode]["Retail Score Relative to National Median"]=lastdfdict[postcode]["Businesses per Hectere"]/allmedianretaildensity
lastdfdict["KT1"]

calculating the median prices and median times for each postcode
405000 KT1 this is the median price for KT1
Done
putting the last df into a df to adjust index and drop columns
Done
putting the last df back into a dict as the last step
Lastdf written to dict
postcode df being written to df
postcode df dict being split to get first postcode
getting local authority for first postcode
6982.758620689655
6982.758620689655
6982.758620689655
6982.758620689655
getting population for local authority
getting deprivation rank for local authority
getting median price for area relative to median price within 2 hrs of work
getting retail counts for each local authority
getting crime counts for local authority
getting area for local authority
getting postcodes with any unknown values
dropping any postcodes from lastdfdict with any NA values
866
17282.0


{'Median Price': 405000,
 'Median Time': 54,
 'local authority': 'Kingston upon Thames',
 'Median Price over Median Area (Kingston Only)': 6982.758620689655,
 'Population': 160060,
 'Deprivation Rank': 24742.0,
 'Postcode Median Price Relative to 2hr Radius Median Price': 2.383053839364519,
 'Postcode Median Time Relative to 2hr Radius Median Time': 0.421875,
 'Retail Count': 1105.0,
 'Crime Count': 12623,
 'Area': 3725.93,
 'Deprivation Score Relative to Nation Median': 1.4316630019673648,
 'Crimes per 1000': 0.07886417593402474,
 'Crime rate relative to National Median': 0.89675824831936,
 'Businesses per Hectere': 0.2965702522591675,
 'Retail Score Relative to National Median': 9.575059750183176}

In [11]:
lastdfdict["E10"]

{'Median Price': 370000,
 'Median Time': 34,
 'local authority': 'Waltham Forest',
 'Median Price over Median Area (Kingston Only)': 0,
 'Population': 258249,
 'Deprivation Rank': 11134.0,
 'Postcode Median Price Relative to 2hr Radius Median Price': 2.1771109149749925,
 'Postcode Median Time Relative to 2hr Radius Median Time': 0.265625,
 'Retail Count': 1360.0,
 'Crime Count': 22794,
 'Area': 3881.02,
 'Deprivation Score Relative to Nation Median': 0.6442541372526328,
 'Crimes per 1000': 0.08826365252140377,
 'Crime rate relative to National Median': 1.003638946174725,
 'Businesses per Hectere': 0.3504233423172259,
 'Retail Score Relative to National Median': 11.313759269470395}

In [12]:
lastdfdict = {key:val for key, val in lastdfdict.items() if lastdfdict[key]["Median Price"]<400000}
for key,val in lastdfdict.items():
    print(key,lastdfdict[key]["Median Price"])

AL1 395000
AL2 230000
AL5 317500
AL6 310000
AL8 275000
AL9 302477
B11 193000
B15 182000
B18 159750
B20 87875
B23 84975
B27 103000
B28 127500
B29 110000
B30 135000
B31 131000
B33 104500
B37 98000
B38 85000
B42 103000
B46 196500
B5 95000
B66 72500
B69 95500
B7 79500
B70 90000
B73 145500
B74 179750
B77 115250
B78 151400
B79 104500
B8 52500
B90 150750
B91 205000
B92 151000
B93 255000
BA1 305000
BA12 127950
BA13 136500
BA14 142000
BA15 185000
BA2 247500
BB1 83500
BB11 116250
BB12 70748
BB2 90000
BB3 80250
BB5 52000
BB7 138750
BD10 105475
BD16 110000
BD17 95000
BD18 106498
BD20 139000
BD21 93000
BH25 235000
BL3 85600
BL4 81500
BL5 107500
BL6 85000
BL7 159500
BN1 285000
BN11 225000
BN12 312500
BN14 197500
BN15 177500
BN16 288000
BN17 177000
BN18 337500
BN2 275000
BN21 192500
BN22 155000
BN24 200000
BN25 206250
BN26 185000
BN3 327500
BN41 260000
BN42 223000
BN43 258500
BN6 310000
BN7 266500
BN9 170000
BR1 328000
BR2 300000
BR3 317500
BR4 190000
BR5 237500
BR6 320000
BR7 252500
BR8 250000
BS22 

In [13]:
maxmedprice=0
for postcode in list(lastdfdict.keys()):
    medprice=lastdfdict[postcode]["Median Price"]
    if medprice>maxmedprice:
        maxmedprice=medprice
print("maxprice",maxmedprice)
maxbusinesses=0
for postcode in list(lastdfdict.keys()):
    businessdens=lastdfdict[postcode]["Businesses per Hectere"]
    if businessdens>maxbusinesses:
        maxbusinesses=businessdens
print("maxbusinesses",maxbusinesses)
maxdeprank=0
for postcode in list(lastdfdict.keys()):
    deprank=lastdfdict[postcode]["Deprivation Rank"]
    if deprank>maxdeprank:
        maxdeprank=deprank
print("maxdep",maxdeprank)
maxcrimerank=0
for postcode in list(lastdfdict.keys()):
    crimerank=lastdfdict[postcode]["Crimes per 1000"]
    if crimerank>maxcrimerank:
        maxcrimerank=crimerank
print("maxcrime",maxcrimerank)
maxtimerank=0
for postcode in list(lastdfdict.keys()):
    timerank=lastdfdict[postcode]["Median Time"]
    if timerank>maxtimerank:
        maxtimerank=timerank
print("maxtime",maxtimerank)

maxprice 399000
maxbusinesses 0.9714345725687881
maxdep 30960.0
maxcrime 0.18665289136780047
maxtime 595


In [27]:
lastdfdict["KT1"]

{'Median Price': 405000,
 'Median Time': 54,
 'local authority': 'Kingston upon Thames',
 'Median Price over Median Area (Kingston Only)': 6982.758620689655,
 'Population': 160060,
 'Deprivation Rank': 24742.0,
 'Postcode Median Price Relative to 2hr Radius Median Price': 1.4516129032258065,
 'Postcode Median Time Relative to 2hr Radius Median Time': 1.0,
 'Retail Count': 1105.0,
 'Crime Count': 11776,
 'Area': 3725.93,
 'Deprivation Score Relative to Nation Median': 1.257726718178121,
 'Crimes per 1000': 0.0735724103461202,
 'Crime rate relative to National Median': 0.8908753249795753,
 'Businesses per Hectere': 0.2965702522591675,
 'Retail Score Relative to National Median': 3.120374254618747}

In [15]:
lastdfdict["KT6"]

{'Median Price': 396375,
 'Median Time': 43,
 'local authority': 'Kingston upon Thames',
 'Median Price over Median Area (Kingston Only)': 6982.758620689655,
 'Population': 160060,
 'Deprivation Rank': 24742.0,
 'Postcode Median Price Relative to 2hr Radius Median Price': 2.3323036187113857,
 'Postcode Median Time Relative to 2hr Radius Median Time': 0.3359375,
 'Retail Count': 1105.0,
 'Crime Count': 12623,
 'Area': 3725.93,
 'Deprivation Score Relative to Nation Median': 1.4316630019673648,
 'Crimes per 1000': 0.07886417593402474,
 'Crime rate relative to National Median': 0.89675824831936,
 'Businesses per Hectere': 0.2965702522591675,
 'Retail Score Relative to National Median': 9.575059750183176}

In [16]:
for postcode in list(lastdfdict.keys()):
    price_score=(lastdfdict[postcode]['Median Price']/maxmedprice)**2
    # smaller is better
    retail_score=(lastdfdict[postcode]["Businesses per Hectere"]/maxbusinesses)
    # bigger is better
    deprivation_score=(lastdfdict[postcode]["Deprivation Rank"]/maxdeprank)**2
    # bigger is better
    crime_score=(lastdfdict[postcode]["Crimes per 1000"]/maxcrimerank)**1.25
    # smaller is better
    #time_score=(lastdfdict[postcode]["Median Time"]/maxtimerank)
    # smaller is better
    lastdfdict[postcode]["Total Score"]=(deprivation_score)/(price_score*crime_score)
    # bigger is better
outputdf=pd.DataFrame.from_dict(data=lastdfdict,orient="index")
outputdf=outputdf[["local authority","Median Price","Median Price over Median Area (Kingston Only)","Businesses per Hectere","Deprivation Rank","Crimes per 1000","Median Time","Total Score"]]
savedf=outputdf.sort_values(by="Total Score",ascending=False)
savedf.to_excel("revised_weighted_savedf_flats_national.xlsx")

In [17]:
savedf

,local authority,Median Price,Median Price over Median Area (Kingston Only),Businesses per Hectere,Deprivation Rank,Crimes per 1000,Median Time,Total Score
CW2,Cheshire East,63500,0.000000,0.010803,23617.5,0.060806,134,93.351920
WN8,West Lancashire,62500,0.000000,0.007997,20421.0,0.051931,538,87.749829
NG34,North Kesteven,89000,0.000000,0.002764,25367.0,0.042156,439,86.660118
LN4,North Kesteven,123500,0.000000,0.002764,25367.0,0.042156,479,45.005478
YO25,East Riding of Yorkshire,91250,0.000000,0.003813,21665.5,0.058672,525,39.780385
NG31,South Kesteven,93750,0.000000,0.004562,23011.0,0.061934,105,39.733426
CW4,Cheshire East,97500,0.000000,0.010803,23617.5,0.060806,155,39.596926
PO14,Fareham,122998,0.000000,0.028888,28052.0,0.056148,125,38.779778
BB7,Ribble Valley,138750,0.000000,0.004962,26132.5,0.041658,521,38.407082
GL11,Stroud,116750,0.000000,0.007677,25145.0,0.051818,423,38.231764


In [63]:
savedfkt=savedf[savedf.index.str.startswith("KT")].sort_values(by="Total Score",ascending=False).head()
savedfsw=savedf[savedf.index.str.startswith("SW")].sort_values(by="Total Score",ascending=False).head()
savedftw=savedf[savedf.index.str.startswith("TW")].sort_values(by="Total Score",ascending=False).head()
savedffinal=pd.concat([savedfkt,savedfsw,savedftw])
savedffinal

,local authority,Median Price,Businesses per Hectere,Deprivation Rank,Crimes per 1000,Median Time,Total Score
KT6,Kingston upon Thames,396375,0.296570,24742.0,0.073572,43,0.922222
KT1,Kingston upon Thames,405000,0.296570,24742.0,0.073572,54,0.718723
KT4,Epsom and Ewell,355000,0.076292,29613.5,0.061250,51,0.402333
KT7,Elmbridge,365000,0.051903,29646.0,0.055297,59,0.262063
KT8,Elmbridge,351000,0.051903,29646.0,0.055297,64,0.251225
SW14,Richmond upon Thames,451500,0.145541,27732.0,0.058538,45,0.634727
SW20,Merton,366250,0.198011,21836.0,0.064209,54,0.489982
SW15,Wandsworth,497500,0.319457,18736.0,0.082584,35,0.482602
SW19,Merton,444500,0.198011,21836.0,0.064209,49,0.444922
SW18,Wandsworth,514250,0.319457,18736.0,0.082584,38,0.430024


In [17]:
savedf["Area"]=""
savedf["Sub-Area"]=""
savedf["Comment"]=""
savedf.loc["KT6","Area"]="Kingston"
savedf.loc["KT6","Sub-Area"]="Surbiton"
savedf.loc["KT6","Sub-Area"]="Surbiton"
savedf.loc["KT1","Area"]="Kingston"
savedf.loc["KT1","Sub-Area"]="Centre"
savedf.loc["SW14","Area"]="Kingston"
savedf.loc["SW14","Sub-Area"]="East Sheen"
savedf.loc["TW2","Area"]="Twickenham"
savedf.loc["TW2","Sub-Area"]="Whitton"
savedf.loc["TW1","Area"]="Twickenham"
savedf.loc["TW1","Sub-Area"]="Center"
savedf.loc["TW11","Area"]="Twickenham"
savedf.loc["TW11","Sub-Area"]="Teddington"
savedf.loc["TW12","Area"]="Twickenham"
savedf.loc["TW12","Sub-Area"]="Hampton"
#savedf.sort_values(by="Total Score",ascending=False)

In [30]:
savedf=savedf[savedf["Area"].str.len()>1]
savedf.sort_values(by="Total Score",ascending=False)

,local authority,Median Price,Businesses per Hectere,Deprivation Rank,Crimes per 1000,Median Time,Total Score,Area,Sub-Area
KT6,Kingston upon Thames,396375,0.296570,24742.0,0.073572,43,0.922222,Kingston,Surbiton
KT1,Kingston upon Thames,405000,0.296570,24742.0,0.073572,54,0.718723,Kingston,Centre
TW2,Richmond upon Thames,399000,0.145541,27732.0,0.058538,50,0.646419,Twickenham,Whitton
SW14,Richmond upon Thames,451500,0.145541,27732.0,0.058538,45,0.634727,Kingston,East Sheen
TW1,Richmond upon Thames,412500,0.145541,27732.0,0.058538,53,0.589871,Twickenham,Center
TW11,Richmond upon Thames,375000,0.145541,27732.0,0.058538,59,0.582873,Twickenham,Teddington
TW12,Richmond upon Thames,364000,0.145541,27732.0,0.058538,61,0.580799,Twickenham,Hampton


In [32]:
savedf.groupby('Area', as_index=False)['Total Score'].mean()

,Area,Total Score
0,Kingston,0.758557
1,Twickenham,0.599990


In [21]:
outputdf=outputdf[outputdf["local authority"]!="City of London"]
outputdf=outputdf[outputdf["local authority"]!="Westminster"]
outputdf[(outputdf["Median Time"]<75)].sort_values(by="Total Score",ascending=False)

,local authority,Median Price,Businesses per Hectere,Deprivation Rank,Crimes per 1000,Median Time,Total Score
SE20,Bromley,252500,0.061945,24247.0,0.072116,27,7.134760
CM13,Brentwood,276750,0.016653,26511.0,0.073341,34,5.260215
CM15,Brentwood,272000,0.016653,26511.0,0.073341,38,4.569036
BR7,Bromley,310000,0.061945,24247.0,0.072116,34,3.711558
SE9,Bromley,300000,0.061945,24247.0,0.072116,37,3.434256
KT19,Epsom and Ewell,265000,0.076292,29613.5,0.061250,57,3.227004
BR6,Bromley,320000,0.061945,24247.0,0.072116,38,2.995137
CM14,Brentwood,275000,0.016653,26511.0,0.073341,50,2.977826
CM23,East Hertfordshire,256250,0.015452,27822.0,0.051356,58,2.918367
GU22,Woking,262500,0.104554,27122.0,0.064326,55,2.896720


In [12]:
outputdf[(outputdf["Postcode Median Price Relative to 2hr Radius Median Price"]<1) & (outputdf["Postcode Median Time Relative to 2hr Radius Median Time"]<1)].sort_values(by="Total Score",ascending=False)["local authority"].to_dict()

{'IG1': 'Redbridge',
 'KT9': 'Kingston upon Thames',
 'CR4': 'Merton',
 'WD19': 'Watford',
 'SE18': 'Greenwich',
 'IG11': 'Barking and Dagenham',
 'SM6': 'Sutton',
 'SM5': 'Sutton',
 'GU21': 'Woking',
 'CR0': 'Croydon',
 'SE2': 'Greenwich',
 'RM9': 'Barking and Dagenham',
 'DA17': 'Bexley',
 'DA8': 'Bexley',
 'SE25': 'Croydon',
 'CR7': 'Croydon',
 'DA15': 'Bexley',
 'TW13': 'Hounslow',
 'UB2': 'Hounslow',
 'DA1': 'Bexley',
 'RM13': 'Havering',
 'N18': 'Enfield',
 'DA16': 'Bexley',
 'EN3': 'Enfield',
 'LU1': 'Luton',
 'DA7': 'Bexley',
 'N9': 'Enfield',
 'BR1': 'Bromley',
 'UB3': 'Hillingdon',
 'RG1': 'Reading',
 'UB7': 'Hillingdon',
 'RM3': 'Havering',
 'SG1': 'Stevenage',
 'SS9': 'Southend-on-Sea',
 'SS0': 'Southend-on-Sea',
 'EN8': 'Broxbourne',
 'EN10': 'Broxbourne',
 'EN11': 'Broxbourne',
 'SS15': 'Basildon',
 'SS16': 'Basildon',
 'CM20': 'Harlow',
 'SS7': 'Castle Point',
 'RH6': 'Reigate and Banstead',
 'RH10': 'Crawley',
 'CM19': 'Harlow',
 'ME4': 'Medway',
 'RM19': 'Thurrock',
 '